# Simulation setup example

This example shows how I use the romscom utilities to simplify input parameter setup and run ROMS simulations.  This script runs the hindcast for the Bering10K domain with the Banas model.

In [2]:
import os
import sys
from datetime import datetime, timedelta
import romscom.romscom as rc

## Baseline parameters

The baseline input parameters are held in a series of YAML files.  These can be considered the starting point for our ROMS application.  From here, we can make adjustments that are simulation-specific.

For this example, I am using input parameter files for the "Bering10K biogeochemical variants app".  The YAML files associated with this ROMS Application can be found on GitHub under the [beringnpz-beringApps](https://github.com/beringnpz/bering-Apps) project.  The  forcing/boundary/etc. files associated with the App are not currently publicly available, meaning that users will need to substit

We start by reading these values into dictionaries.

In [3]:
bapps = "~/Documents/Research/Working/mox_bumblereem/" # KK local-mounted path to /gcratch/bumblereem
# bapps = "/gscratch/bumblereem/"                      # path on mox.hyak.uw.edu
appfol = os.path.join(bapps, "kearney", "bering-Apps/Apps/Bering_with_bio")

ocean   = rc.readparamfile(os.path.join(appfol, "bering_ocean.yaml"), tconvert=True)
station = rc.readparamfile(os.path.join(appfol, "bering_spos.yaml"))
bio     = rc.readparamfile(os.path.join(appfol, "bering_bpar_banas.yaml"))
ice     = rc.readparamfile(os.path.join(appfol, "bering_ipar.yaml"))

## Modifying parameters

### Basic parameters

Dictionary values can be accessed by their key (parameter) name.

In [3]:
# Change tiling from 5-node to 1-node configuration

ocean['NtileJ'] = 5

### Station locations

The station position parameter is a table in the ROMS standard input.  In this toolbox, each station is represented by a 4-element list with the table row values, and the 'POS' value is itself a list of these 4-element lists.

In [4]:
# Adding a station in Unimak Pass

slat = 54.281
slon = -165.03   

if slon < 0:
    slon = slon + 360

# Each new station should be added as a 4-element list: 
# [grid, flag (1=lat/lon, 0=xi/eta), x (lon), y (lat)]

station['POS'].append([1, 1, slon, slat])
station['NSTATION'] = len(station['POS'])
    

### Time variables

When read in with the tconvert flag (as above), time-related variables are converted to datetime and timedelta objects.  This allows you to set things like run duration and archiving time steps independent of the model time step, and without having to do any math.  The values will be translated to their appropriate ROMS unit when converting to standard input format.

In [5]:
# Set end date by adjusting NTIMES

enddate = datetime(2002,3,1)

ocean['NTIMES'] = enddate - ocean['DSTART']

# Set archiving time steps and file size

ocean['NSTA'] = timedelta(hours=6)
ocean['NAVG'] = timedelta(weeks=1)
ocean['NHIS'] = timedelta(weeks=1)
ocean['NDEFAVG'] = timedelta(weeks=10)
ocean['NDEFHIS'] = timedelta(weeks=10)


## Running a simulation

### The basic way

This method simply writes the parameters to files, then calls the ROMS executable.  

In [6]:
simname = "example_sim"

# Write bio, station, and ice parameters to file, and adjust filenames accordingly

bpar = f"{simname}_bpar.in"
ipar = f"{simname}_ipar.in"
spos = f"{simname}_spos.in"

rc.dict2standardin(bio, compress=False, file=bpar)
rc.dict2standardin(ice, compress=False, file=ipar)
rc.dict2standardin(station, compress=False, file=spos)

ocean['BPARNAM'] = bpar
ocean['IPARNAM'] = ipar
ocean['SPOSNAM'] = spos

# Rename output to reflect the sim.  We can do this manually, or use the setoutfilenames function

rc.setoutfilenames(ocean, simname)

# Write parameters to .in file

standinfile  = f"{simname}.in"
standoutfile = f"{simname}.log"
standerrfile = f"{simname}.err"

rc.dict2standardin(ocean, compress=False, file=standinfile)

# Call ROMS executable
# The exact syntax here will depend on your machine and whether you are running in serial, with MPI, or with OpenMP.

mpicmd = "mpirun"
romsexec = "romsM"

romscmd = f"{mpicmd} {romsexec} {standinfile}"

# Below is the syntax I use to call the ROMS process from a python script.  
# Commented out here since exact ROMS setup varies by user and computer.

# with open(standoutfile, 'w') as fout, open(standerrfile, 'w') as ferr:
#     subprocess.run(romscmd, stdout=fout, stderr=ferr)

### The checkpoint way

The `runtodate` routine was written to simplify running and restarting a simulation.  An initial call  to `runtodate` sets up a simulation similar to the above demonstration; however, if the run is stopped for any reason prior to completion, a second call using identical syntax will resume the simulation where it left off.  The routine also includes logic to get past simple blowups by temporarily decreasing the model time step.

In [7]:
## With this method, we still need to set up the "extra" input files manually.

simname = "example_sim"
simdir = "."

# You can place them anywhere, but I like to keep them in the same input folder 
# where the ocean.in files will go

fol = rc.simfolders(simdir, create=True)

bpar = os.path.join(fol['in'], f"{simname}_bpar.in")
ipar = os.path.join(fol['in'], f"{simname}_ipar.in")
spos = os.path.join(fol['in'], f"{simname}_spos.in")

rc.dict2standardin(bio, compress=False, file=bpar)
rc.dict2standardin(ice, compress=False, file=ipar)
rc.dict2standardin(station, compress=False, file=spos)

ocean['BPARNAM'] = bpar
ocean['IPARNAM'] = ipar
ocean['SPOSNAM'] = spos

# Call runtodate routine

romscmd = f"{mpicmd} {romsexec}" # ROMS command shouldn't include standard input file

rc.runtodate(ocean, simdir, simname, enddate=enddate, dryrunflag=True)

WARNING!: Cannot find file ./Input/Bering10K_extended_grid.nc
WARNING!: Cannot find file ./Input/delta_46_his_00168.nc
WARNING!: Cannot find file ./Input/clm_po4.nc
WARNING!: Cannot find file ./Input/2001/roms-cfs-bryphys-N30-2001.nc |
./Input/2002/roms-cfs-bryphys-N30-2002.nc |
./Input/2003/roms-cfs-bryphys-N30-2003.nc |
./Input/2004/roms-cfs-bryphys-N30-2004.nc |
./Input/2005/roms-cfs-bryphys-N30-2005.nc |
./Input/2006/roms-cfs-bryphys-N30-2006.nc
WARNING!: Cannot find file ./Input/2001/roms-cfs-brycarbonate-N30-2001.nc |
./Input/2002/roms-cfs-brycarbonate-N30-2002.nc |
./Input/2003/roms-cfs-brycarbonate-N30-2003.nc |
./Input/2004/roms-cfs-brycarbonate-N30-2004.nc |
./Input/2005/roms-cfs-brycarbonate-N30-2005.nc |
./Input/2006/roms-cfs-brycarbonate-N30-2006.nc
WARNING!: Cannot find file ./Input/roms-brybio-N30.nc
WARNING!: Cannot find file /gscratch/bumblereem/bering10k/input/generic/tides_OTBS.nc
WARNING!: Cannot find file ./Input/2002/roms-cfs-atmos-Pair-2002.nc |
./Input/2003/roms

SystemExit: 

/Users/kakearney/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
test = ["mpirun"]
test.extend(["./romsM"])

test + ["another"]

['mpirun', './romsM', 'another']

In [16]:
test = ["a","b","c"]
[x for x in test if x != "b"]

['a', 'c']